# AUMCdb Data Extraction

**Structuur:**
- `../cfg_params_unified.sql` - Gedeelde config (voor AUMCdb én MIMIC)
- `../AUMCdb_shared_sql/` - AUMCdb-specifieke SQL templates
- `../MIMIC_shared_sql/` - MIMIC-specifieke SQL templates

**Pas alleen de 5 parameters in CONFIG aan:**
- `DATABASE`: "AUMCdb" of "MIMIC"
- `OBS_DAYS`: Observatie window in dagen
- `GRID_STEP_HOURS`: Grid step in uren  
- `INCLUSION`: "k2" of "k3" (KDIGO stage)
- `MIN_ICU_HOURS`: Minimale ICU verblijfsduur

# CONFIG - Pas dit aan per cohort

In [ ]:
# =============================================================================
# COHORT CONFIGURATION - ALLEEN DIT AANPASSEN
# =============================================================================
# Database = MIMIC / AUMCdb
# OBS days = 3/5/7/14/28
# GRID = 8/12/24
# INCLUSION = kdigo2/kdigo3

# Base parameters
DATABASE = "MIMIC"          # "AUMCdb" of "MIMIC"
OBS_DAYS = 3                 # Observation window in dagen
GRID_STEP_HOURS = 8         # Grid step in uren
INCLUSION = "kdigo2"             # "k2" of "k3" (KDIGO stage)
MIN_ICU_HOURS = 24           # Minimale ICU verblijfsduur in uren

# Auto-generated settings
REGION = "EU" if DATABASE == "AUMCdb" else "US"  # AUMCdb=EU, MIMIC=US
DATASET_NAME = f"derived_{DATABASE}_obs{OBS_DAYS}_grid{GRID_STEP_HOURS}_{INCLUSION}"
OUTPUT_BASE = r"C:\Users\karel\Desktop\data\Thesis\Data"

CONFIG = {
    # BigQuery settings
    "PROJECT": "windy-forge-475207-e3",
    "DATASET": DATASET_NAME,
    "REGION": REGION,
    
    # Cohort parameters
    "COHORT_NAME": f"{DATABASE} - {INCLUSION}, ({OBS_DAYS}d, {GRID_STEP_HOURS}h)",
    "OBS_DAYS": OBS_DAYS,
    "GRID_STEP_HOURS": GRID_STEP_HOURS,
    "INCLUSION": INCLUSION,  # "k3" -> "kdigo3"
    "MIN_ICU_HOURS": MIN_ICU_HOURS,
    
    # Output settings
    "OUTPUT_DIR": f"{OUTPUT_BASE}\\{DATASET_NAME}\\Full_dataset",
}

print(f"{'='*60}")
print(f"Cohort:  {CONFIG['COHORT_NAME']}")
print(f"Dataset: {CONFIG['DATASET']}")
print(f"Region:  {CONFIG['REGION']}")
print(f"Min ICU: {CONFIG['MIN_ICU_HOURS']} uur")
print(f"Output:  {CONFIG['OUTPUT_DIR']}")
print(f"{'='*60}")

Cohort:  MIMIC - kdigo2, (3d, 8h)
Dataset: derived_MIMIC_obs3_grid8_kdigo2
Region:  US
Min ICU: 24 uur
Output:  C:\Users\karel\Desktop\data\Thesis\Data\derived_MIMIC_obs3_grid8_kdigo2\Full_dataset


# Setup

In [161]:
from google.cloud import bigquery
import pandas as pd
from pathlib import Path
from datetime import datetime, timezone

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 200)

# Paths
BASE_DIR = Path("..")
CONFIG_FILE = BASE_DIR / "cfg_params_unified.sql"       # Gedeeld config bestand
SQL_DIR = BASE_DIR / f"{DATABASE}_shared_sql"           # Database-specifieke SQL

# BigQuery client (region based on database)
client = bigquery.Client(project=CONFIG["PROJECT"], location=CONFIG["REGION"])

In [162]:
# Create dataset (region based on database: AUMCdb=EU, MIMIC=US)
dataset_id = f"{CONFIG['PROJECT']}.{CONFIG['DATASET']}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = CONFIG["REGION"]
client.create_dataset(dataset, exists_ok=True)
print(f"Dataset {CONFIG['DATASET']} exists ({CONFIG['REGION']} region)")

Dataset derived_MIMIC_obs3_grid8_kdigo2 exists (US region)


In [163]:
def run_sql(sql_path: Path):
    """Load SQL template, replace placeholders, and execute."""
    sql = sql_path.read_text(encoding="utf-8")
    
    # Replace all placeholders from CONFIG
    for key, value in CONFIG.items():
        placeholder = f"${{{key}}}"
        sql = sql.replace(placeholder, str(value))
    
    # Execute
    job = client.query(sql)
    job.result()  # wait for completion
    print(f"{sql_path.name} - Done at {datetime.now(timezone.utc).astimezone().strftime('%H:%M:%S')}")

def get_sort_key(filename: str) -> tuple:
    """Extract SQL number for sorting (e.g., SQL5_1 -> (5, 1), SQL14 -> (14, 0))."""
    import re
    match = re.search(r'SQL(\d+)(?:_(\d+))?', filename)
    if match:
        main = int(match.group(1))
        sub = int(match.group(2)) if match.group(2) else 0
        return (main, sub)
    return (999, 0)

def run_all_sql():
    """Run all SQL files in correct order."""
    # Get all SQL files in the database-specific folder, sorted by SQL number
    sql_files = sorted(SQL_DIR.glob("*.sql"), key=lambda f: get_sort_key(f.name))
    
    print(f"Running {len(sql_files) + 1} SQL files for {DATABASE}...\n")
    
    # 1. Run shared config first
    run_sql(CONFIG_FILE)
    
    # 2. Run database-specific SQL files in order
    for sql_file in sql_files:
        run_sql(sql_file)
    
    print(f"\nAll done!")

# Run All SQL

In [164]:
run_all_sql()

Running 21 SQL files for MIMIC...

cfg_params_unified.sql - Done at 10:20:52
SQL1_utils_MIMIC.sql - Done at 10:21:23


KeyboardInterrupt: 

# Action Frequency Analysis

# Summary

In [ ]:
# Load data and calculate summary statistics
query = f"""
SELECT * 
FROM `{CONFIG['PROJECT']}.{CONFIG['DATASET']}.grid_master_all_features`
"""
df = client.query(query).to_dataframe()

# Basic stats
total_rows = len(df)
total_actions = df['action_rrt'].sum()
action_freq = (total_actions / total_rows) * 100
n_stays = df['visit_occurrence_id'].nunique()
n_persons = df['person_id'].nunique()

# Summary table
print(f"\n{'='*60}")
print(f"SUMMARY - {CONFIG['COHORT_NAME']}")
print(f"{'='*60}")
print(f"Dataset:          {CONFIG['DATASET']}")
print(f"Inclusion:        {CONFIG['INCLUSION']}")
print(f"Obs window:       {CONFIG['OBS_DAYS']} days")
print(f"Grid step:        {CONFIG['GRID_STEP_HOURS']} hours")
print(f"Unique stays:     {n_stays:,}")
print(f"Unique persons:   {n_persons:,}")
print(f"Total rows:       {total_rows:,}")
print(f"Action frequency: {action_freq:.2f}%")
print(f"{'='*60}")

# Save General Features - Local & BigQuery

In [ ]:
import json

# Calculate terminal events distribution per grid step (hours since t0)
df['hours_since_t0'] = (df['grid_ts'] - df['t0']).dt.total_seconds() / 3600
terminal_df = df[df['is_terminal_step'] == True].copy()

# Terminal events per grid step
terminal_dist = terminal_df.groupby(['hours_since_t0', 'terminal_event']).size().unstack(fill_value=0)
terminal_dist_dict = terminal_dist.to_dict(orient='index')

# Convert keys to strings for JSON compatibility (keys are hours since t0, i.e. grid steps)
terminal_dist_json = {str(int(k)): v for k, v in terminal_dist_dict.items()}

# Create general features dictionary
general_features = {
    "dataset_name": DATASET_NAME,
    "database": DATABASE,
    "inclusion": INCLUSION,
    "obs_days": OBS_DAYS,
    "grid_step_hours": GRID_STEP_HOURS,
    "min_icu_hours": MIN_ICU_HOURS,
    "unique_stays": int(n_stays),
    "unique_persons": int(n_persons),
    "total_rows": int(total_rows),
    "total_actions": int(total_actions),
    "action_frequency_pct": round(action_freq, 4),
    "terminal_events_total": terminal_df['terminal_event'].value_counts().to_dict(),
    "terminal_events_by_grid_step": terminal_dist_json,  # keys = hours since t0 (grid steps)
    "extraction_timestamp": datetime.now(timezone.utc).isoformat()
}

# Save locally as JSON
output_dir = Path(CONFIG["OUTPUT_DIR"])
output_dir.mkdir(parents=True, exist_ok=True)
json_path = output_dir / f"{DATASET_NAME}_general_features.json"
with open(json_path, 'w') as f:
    json.dump(general_features, f, indent=2)
print(f"General features saved locally: {json_path}")

# Save to BigQuery as table
features_df = pd.DataFrame([{
    "dataset_name": DATASET_NAME,
    "database": DATABASE,
    "inclusion": INCLUSION,
    "obs_days": OBS_DAYS,
    "grid_step_hours": GRID_STEP_HOURS,
    "min_icu_hours": MIN_ICU_HOURS,
    "unique_stays": n_stays,
    "unique_persons": n_persons,
    "total_rows": total_rows,
    "total_actions": total_actions,
    "action_frequency_pct": round(action_freq, 4),
    "terminal_death": int(terminal_df[terminal_df['terminal_event'] == 'death'].shape[0]),
    "terminal_discharge": int(terminal_df[terminal_df['terminal_event'] == 'discharge'].shape[0]),
    "terminal_rrt_start": int(terminal_df[terminal_df['terminal_event'] == 'rrt_start'].shape[0]),
    "terminal_window_end": int(terminal_df[terminal_df['terminal_event'] == 'window_end'].shape[0]) if 'window_end' in terminal_df['terminal_event'].values else 0,
    "extraction_timestamp": datetime.now(timezone.utc)
}])

# Upload to BigQuery
table_id = f"{CONFIG['PROJECT']}.{CONFIG['DATASET']}.general_features"
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
client.load_table_from_dataframe(features_df, table_id, job_config=job_config).result()
print(f"General features saved to BigQuery: {table_id}")

# Display summary
print(f"\n{'='*60}")
print("GENERAL FEATURES")
print(f"{'='*60}")
for k, v in general_features.items():
    if k not in ['terminal_events_by_grid_step']:
        print(f"{k}: {v}")
print(f"{'='*60}")

# Extraction

In [ ]:
# Download final dataset from BigQuery
query = f"""
SELECT * 
FROM `{CONFIG['PROJECT']}.{CONFIG['DATASET']}.grid_master_all_features`
"""

df_final = client.query(query).to_dataframe()

# Save locally as parquet
output_dir = Path(CONFIG["OUTPUT_DIR"])
output_dir.mkdir(parents=True, exist_ok=True)
output_path = output_dir / "aumc_rrt_raw.parquet"
df_final.to_parquet(output_path)

print(f"Dataset saved: {output_path}")
print(f"Shape: {df_final.shape}")

Dataset saved: C:\Users\karel\Desktop\data\Thesis\Data\derived_MIMIC_obs5_grid12_kdigo3\Full_dataset\aumc_rrt_raw.parquet
Shape: (110893, 132)


In [ ]:
# Final dataset summary
print(f"Total rows: {len(df_final):,}")
print(f"Total columns: {len(df_final.columns)}")
print(f"\nUnique stays: {df_final['visit_occurrence_id'].nunique():,}")
print(f"Unique subjects: {df_final['person_id'].nunique():,}")
print(f"\nTerminal events:")
print(df_final['terminal_event'].value_counts())

Total rows: 110,893
Total columns: 132

Unique stays: 11,879
Unique subjects: 10,047

Terminal events:
terminal_event
discharge    8085
rrt_start    2178
death        1616
Name: count, dtype: int64
